In [110]:
import pandas as pd
import numpy as np
from cfbd_api import play_by_play



#grabbing all play by play data from the 2022 season for big ten teams
big_ten = play_by_play()


Minnesota
Wisconsin
Ohio State
Penn State
Michigan
Michigan State
Nebraska
Northwestern
Illinois
Indiana
Iowa
Maryland
Purdue
Rutgers


In [111]:
pd.set_option('mode.chained_assignment', None)

big_ten['yards_gained_percentage'] = big_ten['yards_gained'] / big_ten['distance']

values = [1,1,1,3,4,5,-1,-6,(-((99 - big_ten['yard_line']) * (2.6/99)) - 1.4)]

conditions = [(big_ten['down'] == 1) & (big_ten['yards_gained_percentage'] >= 0.45) & (big_ten['yards_gained'] < 10), 
            (big_ten['down'] == 2) & (big_ten['yards_gained_percentage'] >= 0.60) & (big_ten['yards_gained'] < 10),
            (big_ten['down'] == 3) | (big_ten['down'] == 4) & (big_ten['yards_gained_percentage'] >= 1) & (big_ten['yards_gained'] < 10),
            (big_ten['yards_gained_percentage'] > 1) & (big_ten['yards_gained'] >= 10) & (big_ten['yards_gained'] < 20),
            (big_ten['yards_gained_percentage'] > 1) & (big_ten['yards_gained'] >= 20) & (big_ten['yards_gained'] < 40),
            (big_ten['yards_gained_percentage'] > 1) & (big_ten['yards_gained'] >= 40),
            (big_ten['yards_gained'] <= -3),
            (big_ten['period'] != 4) & (big_ten['minutes'] > 2) & big_ten['play_type'].str.contains('Interception'),
            (big_ten['play_type'].str.contains('Fumble'))]

big_ten['success_points'] = np.select(conditions, values, default=0)
big_ten['success_points'].unique()

array([ 0.        , -1.        ,  1.        ,  3.        ,  4.        ,
       -2.87070707,  5.        , -6.        , -1.53131313, -1.87272727,
       -3.34343434, -2.84444444, -3.42222222, -3.0020202 , -2.24040404,
       -3.31717172, -1.92525253, -2.52929293, -3.55353535, -2.03030303,
       -3.29090909, -2.73939394, -1.82020202, -3.5010101 , -1.4       ,
       -2.76565657, -3.21212121, -2.79191919, -1.97777778, -2.68686869,
       -3.08080808, -2.31919192, -2.29292929, -2.13535354, -2.47676768,
       -1.66262626, -3.13333333])

In [112]:
values = [1, 0]
conditions = [(big_ten['success_points'] <= 0), (big_ten['success_points'] > 0)]

big_ten['defense_stopped'] = np.select(conditions, values)
big_ten['defense_stopped'].head()

0    1
1    1
2    1
3    1
4    0
Name: defense_stopped, dtype: int64

In [113]:
big_ten_defense = big_ten[big_ten['down'] > 0]
big_ten_defense_rush = big_ten_defense[big_ten_defense['play_type'].str.contains('Rush')]
big_ten_defense_pass = big_ten_defense[big_ten_defense['play_type'].str.contains('Pass')]

defense_dvoa_rush = big_ten_defense_rush.groupby(['down', 'yard_line'])['success_points'].agg(['mean', 'count']).reset_index()
defense_dvoa_rush.head()

defense_dvoa_pass = big_ten_defense_pass.groupby(['down', 'yard_line'])['success_points'].agg(['mean', 'count']).reset_index()
defense_dvoa_pass.head()

,down,yard_line,mean,count
0,1,1,0.285714,7
1,1,2,-1.000000,2
2,1,3,0.000000,1
3,1,4,0.600000,10
4,1,5,0.200000,5


In [114]:
big_ten_offense = big_ten[big_ten['down'] > 0]
big_ten_offense_rush = big_ten_offense[big_ten_offense['play_type'].str.contains('Rush')]
big_ten_offense_pass = big_ten_offense[big_ten_offense['play_type'].str.contains('Pass')]

offense_dvoa_rush = big_ten_offense_rush.groupby(['down', 'yard_line'])['success_points'].agg(['mean', 'count']).reset_index()
offense_dvoa_rush.head()

offense_dvoa_pass = big_ten_offense_pass.groupby(['down', 'yard_line'])['success_points'].agg(['mean', 'count']).reset_index()
offense_dvoa_pass.head()

,down,yard_line,mean,count
0,1,1,0.285714,7
1,1,2,-1.000000,2
2,1,3,0.000000,1
3,1,4,0.600000,10
4,1,5,0.200000,5


In [115]:
from sklearn.preprocessing import StandardScaler
from cfbd_api import BIG_TEN_TEAMS
conditions = []
values = []

for row in defense_dvoa_rush.to_dict('records'):
    values.append(row['mean'])
    conditions.append((big_ten_defense_rush["yard_line"] == row['yard_line']) & (big_ten_defense_rush["down"] == row['down']) & (big_ten_defense_rush["play_type"].str.contains('Rush')))

big_ten_defense_rush["success_points_average"] = np.select(conditions, values)
big_ten_defense_rush["voa"] = big_ten_defense_rush["success_points"] / big_ten_defense_rush["success_points_average"]

defense_adjustment = big_ten_defense_rush.groupby(['down', 'distance', 'defense'])['defense_stopped'].agg(['sum','count']).reset_index()
defense_adjustment['percentage'] = defense_adjustment['sum'] / defense_adjustment.groupby(['down', 'distance', 'defense'])["count"].transform('sum')

conditions = []
values = []

for row in defense_adjustment.to_dict('records'):
    values.append(row['percentage'])
    conditions.append((big_ten_defense_rush["distance"] == row['distance']) & (big_ten_defense_rush["down"] == row['down']) & (big_ten_defense_rush["defense"] == row['defense']))

big_ten_defense_rush['defense_stop_percentage'] = np.select(conditions, values)
big_ten_defense_rush['adjusted_voa'] = big_ten_defense_rush['voa'] - big_ten_defense_rush['defense_stop_percentage']
big_ten_defense_rush.replace([np.inf, -np.inf], np.nan, inplace=True)
big_ten_defense_rush.dropna(subset=['adjusted_voa'], how='all', inplace=True)
big_ten_defense_rush = big_ten_defense_rush[big_ten_defense_rush['defense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
big_ten_defense_rush['dvoa'] = scaler.fit_transform(big_ten_defense_rush[['adjusted_voa']]).flatten()
big_ten_defense_rush[['voa', 'defense_stop_percentage', 'adjusted_voa', 'dvoa']].head()

big_ten_defense_rush_overall_summary = big_ten_defense_rush.groupby(['defense'])['dvoa'].agg(['mean']).reset_index()
big_ten_defense_rush_overall_summary = big_ten_defense_rush_overall_summary.sort_values(by='mean')

big_ten_defense_rush_down_distance_summary = big_ten_defense_rush.groupby(['defense', 'down', 'distance'])['dvoa'].agg(['mean']).reset_index()
big_ten_defense_rush_down_distance_summary = big_ten_defense_rush_down_distance_summary.sort_values(by='mean')
big_ten_defense_rush_down_distance_summary.head()

,defense,down,distance,mean
116,Iowa,2,16,-2.416702
138,Maryland,1,1,-2.360028
161,Maryland,2,15,-1.964469
343,Nebraska,2,14,-1.950857
232,Michigan State,1,6,-1.944360


In [116]:
conditions = []
values = []

for row in defense_dvoa_pass.to_dict('records'):
    values.append(row['mean'])
    conditions.append((big_ten_defense_pass["yard_line"] == row['yard_line']) & (big_ten_defense_pass["down"] == row['down']) & (big_ten_defense_pass["play_type"].str.contains('Pass')))

big_ten_defense_pass["success_points_average"] = np.select(conditions, values)
big_ten_defense_pass["voa"] = big_ten_defense_pass["success_points"] / big_ten_defense_pass["success_points_average"]

defense_adjustment = big_ten_defense_pass.groupby(['down', 'distance', 'defense'])['defense_stopped'].agg(['sum','count']).reset_index()
defense_adjustment.loc[:, ('percentage')] = defense_adjustment['sum'] / defense_adjustment.groupby(['down', 'distance', 'defense'])["count"].transform('sum')

conditions = []
values = []

for row in defense_adjustment.to_dict('records'):
    values.append(row['percentage'])
    conditions.append((big_ten_defense_pass["distance"] == row['distance']) & (big_ten_defense_pass["down"] == row['down']) & (big_ten_defense_pass["defense"] == row['defense']))

big_ten_defense_pass['defense_stop_percentage'] = np.select(conditions, values)
big_ten_defense_pass['adjusted_voa'] = big_ten_defense_pass['voa'] - big_ten_defense_pass['defense_stop_percentage']
big_ten_defense_pass.replace([np.inf, -np.inf], np.nan, inplace=True)
big_ten_defense_pass.dropna(subset=['adjusted_voa'], how='all', inplace=True)
big_ten_defense_pass = big_ten_defense_pass[big_ten_defense_pass['defense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
big_ten_defense_pass['dvoa'] = scaler.fit_transform(big_ten_defense_pass[['adjusted_voa']]).flatten()

big_ten_defense_pass_overall_summary = big_ten_defense_pass.groupby(['defense'])['dvoa'].agg(['mean']).reset_index()
big_ten_defense_pass_overall_summary = big_ten_defense_pass_overall_summary.sort_values(by='mean')


big_ten_defense_pass_down_distance_summary = big_ten_defense_pass.groupby(['defense', 'down', 'distance'])['dvoa'].agg(['mean']).reset_index()
big_ten_defense_pass_down_distance_summary = big_ten_defense_pass_down_distance_summary.sort_values(by='mean')

big_ten_defense_pass_down_distance_summary.head()

,defense,down,distance,mean
194,Maryland,4,1,-1.033567
259,Michigan State,1,17,-0.884898
573,Purdue,2,24,-0.868969
68,Indiana,2,8,-0.632693
172,Maryland,2,15,-0.502604


In [122]:
conditions = []
values = []

for row in offense_dvoa_rush.to_dict('records'):
    values.append(row['mean'])
    conditions.append((big_ten_offense_rush["yard_line"] == row['yard_line']) & (big_ten_offense_rush["down"] == row['down']) & (big_ten_offense_rush["play_type"].str.contains('Rush')))

big_ten_offense_rush["success_points_average"] = np.select(conditions, values)
big_ten_offense_rush["voa"] = big_ten_offense_rush["success_points"] / big_ten_offense_rush["success_points_average"]

defense_adjustment = big_ten_offense_rush.groupby(['down', 'distance', 'defense'])['defense_stopped'].agg(['sum','count']).reset_index()
defense_adjustment.loc[:, ('percentage')] = defense_adjustment['sum'] / defense_adjustment.groupby(['down', 'distance', 'defense'])["count"].transform('sum')

conditions = []
values = []

for row in defense_adjustment.to_dict('records'):
    values.append(row['percentage'])
    conditions.append((big_ten_offense_rush["distance"] == row['distance']) & (big_ten_offense_rush["down"] == row['down']) & (big_ten_offense_rush["defense"] == row['defense']))

big_ten_offense_rush['defense_stop_percentage'] = np.select(conditions, values)
big_ten_offense_rush['adjusted_voa'] = big_ten_offense_rush['voa'] - big_ten_offense_rush['defense_stop_percentage']
big_ten_offense_rush.replace([np.inf, -np.inf], np.nan, inplace=True)
big_ten_offense_rush.dropna(subset=['adjusted_voa'], how='all', inplace=True)
big_ten_offense_rush = big_ten_offense_rush[big_ten_offense_rush['offense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
big_ten_offense_rush['dvoa'] = scaler.fit_transform(big_ten_offense_rush[['adjusted_voa']]).flatten()

big_ten_offense_rush_overall_summary = big_ten_offense_rush.groupby(['offense'])['dvoa'].agg(['mean']).reset_index()
big_ten_offense_rush_overall_summary = big_ten_offense_rush_overall_summary.sort_values(by='mean', ascending=False)

big_ten_offense_rush_down_distance_summary = big_ten_offense_rush.groupby(['offense', 'down', 'distance'])['dvoa'].agg(['mean']).reset_index()
big_ten_offense_rush_down_distance_summary = big_ten_offense_rush_down_distance_summary.sort_values(by='mean', ascending=False)
big_ten_offense_rush_down_distance_summary.head()

,offense,down,distance,mean
609,Wisconsin,1,14,3.251559
627,Wisconsin,2,19,2.300813
211,Michigan,2,13,2.176619
10,Illinois,1,15,1.749925
31,Illinois,2,22,1.585812


In [123]:
conditions = []
values = []

for row in offense_dvoa_pass.to_dict('records'):
    values.append(row['mean'])
    conditions.append((big_ten_offense_pass["yard_line"] == row['yard_line']) & (big_ten_offense_pass["down"] == row['down']) & (big_ten_offense_pass["play_type"].str.contains('Pass')))

big_ten_offense_pass["success_points_average"] = np.select(conditions, values)
big_ten_offense_pass["voa"] = big_ten_offense_pass["success_points"] / big_ten_offense_pass["success_points_average"]

defense_adjustment = big_ten_offense_pass.groupby(['down', 'distance', 'defense'])['defense_stopped'].agg(['sum','count']).reset_index()
defense_adjustment.loc[:, ('percentage')] = defense_adjustment['sum'] / defense_adjustment.groupby(['down', 'distance', 'defense'])["count"].transform('sum')

conditions = []
values = []

for row in defense_adjustment.to_dict('records'):
    values.append(row['percentage'])
    conditions.append((big_ten_offense_pass["distance"] == row['distance']) & (big_ten_offense_pass["down"] == row['down']) & (big_ten_offense_pass["defense"] == row['defense']))

big_ten_offense_pass['defense_stop_percentage'] = np.select(conditions, values)
big_ten_offense_pass['adjusted_voa'] = big_ten_offense_pass['voa'] - big_ten_offense_pass['defense_stop_percentage']
big_ten_offense_pass.replace([np.inf, -np.inf], np.nan, inplace=True)
big_ten_offense_pass.dropna(subset=['adjusted_voa'], how='all', inplace=True)
big_ten_offense_pass = big_ten_offense_pass[big_ten_offense_pass['offense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
big_ten_offense_pass['dvoa'] = scaler.fit_transform(big_ten_offense_pass[['adjusted_voa']]).flatten()
big_ten_offense_pass[['voa', 'defense_stop_percentage', 'adjusted_voa', 'dvoa']].head()


big_ten_offense_pass_overall_summary = big_ten_offense_pass.groupby(['offense'])['dvoa'].agg(['mean']).reset_index()
big_ten_offense_pass_overall_summary = big_ten_offense_pass_overall_summary.sort_values(by='mean', ascending=False)

big_ten_offense_pass_down_distance_summary = big_ten_offense_pass.groupby(['offense', 'down', 'distance'])['dvoa'].agg(['mean']).reset_index()
big_ten_offense_pass_down_distance_summary = big_ten_offense_pass_down_distance_summary.sort_values(by='mean', ascending=False)
big_ten_offense_pass_down_distance_summary.head()


,offense,down,distance,mean
637,Wisconsin,2,1,3.289115
499,Penn State,2,24,1.533252
165,Maryland,1,25,1.423745
482,Penn State,1,18,1.310171
224,Michigan,2,12,1.191435


In [124]:

overall_offense = pd.concat([big_ten_offense_pass,big_ten_offense_rush])

overall_offense = overall_offense[overall_offense['offense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
overall_offense['dvoa'] = scaler.fit_transform(overall_offense[['adjusted_voa']]).flatten()


overall_offense_summary = overall_offense.groupby(['offense'])['dvoa'].agg(['mean']).reset_index()
overall_offense_summary = overall_offense_summary.sort_values(by='mean', ascending=False)
overall_offense_summary.head()

,offense,mean
9,Ohio State,0.108451
6,Minnesota,0.079272
4,Michigan,0.078816
10,Penn State,0.023735
13,Wisconsin,0.020489


In [125]:
overall_defense = pd.concat([big_ten_defense_pass,big_ten_defense_rush])

overall_defense = overall_defense[overall_defense['offense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
overall_defense['dvoa'] = scaler.fit_transform(overall_defense[['adjusted_voa']]).flatten()
overall_defense[['voa', 'defense_stop_percentage', 'adjusted_voa', 'dvoa']].head()


overall_defense_summary = overall_defense.groupby(['defense'])['dvoa'].agg(['mean']).reset_index()
overall_defense_summary = overall_defense_summary.sort_values(by='mean')
overall_defense_summary.head()

,defense,mean
0,Illinois,-0.068861
4,Michigan,-0.052281
9,Ohio State,-0.035829
2,Iowa,-0.030811
13,Wisconsin,-0.026530


In [126]:
with pd.ExcelWriter('big_ten_dvoa.xlsx') as writer:
    overall_offense_summary.to_excel(writer,'overall_offense', index=False)
    overall_defense_summary.to_excel(writer,'overall_defense', index=False)
    big_ten_offense_pass_overall_summary.to_excel(writer,'offense_pass_overall', index=False)
    big_ten_offense_pass_down_distance_summary.to_excel(writer,'offense_pass_down_and_distance', index=False)
    big_ten_offense_rush_overall_summary.to_excel(writer,'offense_rush_overall', index=False)
    big_ten_offense_rush_down_distance_summary.to_excel(writer,'offense_rush_down_and_distance', index=False)
    big_ten_defense_pass_overall_summary.to_excel(writer,'defense_pass_overall', index=False)
    big_ten_defense_pass_down_distance_summary.to_excel(writer,'defense_pass_down_and_distance', index=False)
    big_ten_defense_rush_overall_summary.to_excel(writer,'defense_rush_overall', index=False)
    big_ten_defense_rush_down_distance_summary.to_excel(writer,'defense_rush_down_and_distance', index=False)
    